<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/02_sintaxis_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [9]:
! pip install polars -U

# Lazy & Collect

In [1]:
import polars as pl

Creemos un dataframe que se conporte a nivel `sintaxis` como `lazy` aunque realmente las operaciones de creacion ya han sido ejecutadas.  
En estricto sentido peudes leer los datos como `lazy` desde el disco, pero para entender como funciona haremos un mock.

In [2]:
# Crear un DataFrame con evaluación perezosa
lazy_df = pl.DataFrame({
    "nombre": ["Ana", "Luis", "Marta"],
    "edad": [25, 32, 18],
    "ciudad": ["Madrid", "Barcelona", "Sevilla"]
}).lazy()


In [10]:
# Seleccionar solo la columna 'nombre'
seleccion = lazy_df.select("nombre")


Que esta haciendo el `select`?

`collect` ejectuta todas las operaciones lazy que no han sido ejecutadas hasta es momento. Digamos que es un comando que indica a `polars` que es momento de ejecutar.

In [9]:
# Ejecutar y mostrar los resultados
print(seleccion.collect())

shape: (3, 1)
┌────────┐
│ nombre │
│ ---    │
│ str    │
╞════════╡
│ Ana    │
│ Luis   │
│ Marta  │
└────────┘


# Filtrado y Operaciones Columna

In [8]:
# Filtrar el DataFrame donde la edad es mayor a 20
filtrado = lazy_df.filter(pl.col("edad") > 20)
filtrado.collect()

shape: (2, 3)
┌────────┬──────┬───────────┐
│ nombre ┆ edad ┆ ciudad    │
│ ---    ┆ ---  ┆ ---       │
│ str    ┆ i64  ┆ str       │
╞════════╪══════╪═══════════╡
│ Ana    ┆ 25   ┆ Madrid    │
│ Luis   ┆ 32   ┆ Barcelona │
└────────┴──────┴───────────┘

Que esta haciendo `filter`?  
Que esta haciendo `pl.col`?

In [12]:
# Crear una nueva columna que sea la longitud del nombre
nueva_columna = filtrado.with_columns(pl.col("nombre").str.len_chars().alias("longitud_nombre"))

Que esta haciendo `with_columns`?  
Por que aparece un `pl.col`?

In [13]:
# Ejecutar y mostrar los resultados
print(nueva_columna.collect())

shape: (2, 4)
┌────────┬──────┬───────────┬─────────────────┐
│ nombre ┆ edad ┆ ciudad    ┆ longitud_nombre │
│ ---    ┆ ---  ┆ ---       ┆ ---             │
│ str    ┆ i64  ┆ str       ┆ u32             │
╞════════╪══════╪═══════════╪═════════════════╡
│ Ana    ┆ 25   ┆ Madrid    ┆ 3               │
│ Luis   ┆ 32   ┆ Barcelona ┆ 4               │
└────────┴──────┴───────────┴─────────────────┘


$$

$$

# Operaciones de agregacion

In [ ]:
# Agrupar por 'ciudad' y calcular la edad promedio
agregacion = lazy_df.group_by("ciudad").agg([
    pl.col("edad").mean().alias("edad_promedio")
])

Que esta haciendo el `agg` en el `groupby`?  
Que esta haciendo el `alias`?

In [16]:
# Ejecutar y mostrar los resultados
print(agregacion.collect())

shape: (3, 2)
┌───────────┬───────────────┐
│ ciudad    ┆ edad_promedio │
│ ---       ┆ ---           │
│ str       ┆ f64           │
╞═══════════╪═══════════════╡
│ Sevilla   ┆ 18.0          │
│ Barcelona ┆ 32.0          │
│ Madrid    ┆ 25.0          │
└───────────┴───────────────┘


# Combinar Operaciones

In [18]:
# Combinar filtrado, agregación y selección de columnas
combinado = lazy_df.filter(pl.col("edad") > 20).group_by("ciudad").agg([
    pl.col("edad").mean().alias("edad_promedio")
]).select(["ciudad", "edad_promedio"])


In [19]:
# Ejecutar y mostrar los resultados
print(combinado.collect())


shape: (2, 2)
┌───────────┬───────────────┐
│ ciudad    ┆ edad_promedio │
│ ---       ┆ ---           │
│ str       ┆ f64           │
╞═══════════╪═══════════════╡
│ Barcelona ┆ 32.0          │
│ Madrid    ┆ 25.0          │
└───────────┴───────────────┘
